In [4]:
import re
import networkx as nx
from pathlib import Path
from networkx.drawing.nx_agraph import graphviz_layout

In [5]:
from flinspect.gen_dependency_graph import gen_dependency_graph

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
build_dirs = [
    "/glade/work/altuntas/turbo-stack/bin/flang_ptree/FMS/",
    "/glade/work/altuntas/turbo-stack/bin/flang_ptree/MOM6/"
]

In [8]:
G = gen_dependency_graph(build_dirs)

In [9]:
pos = nx.nx_agraph.graphviz_layout(G, prog='fdp')

In [10]:
from pyvis.network import Network

In [11]:
def get_color(G, node):
    source_path = G.nodes[node].get('source_path', '')    
    
    node_color, edge_color = "grey", "lightgrey"
    if "/config_src/infra" in source_path:
        node_color, edge_color = "darkred", "red"    
    elif "/MARBL/" in source_path:
        node_color, edge_color = "purple", "orchid"
    elif "/CVMix-src/" in source_path:
        node_color, edge_color = "chocolate", "sandybrown"
    elif "/GSW-Fortran/" in source_path:
        node_color, edge_color = "royalblue", "cornflowerblue"
    elif "/MOM6/" in source_path:
        node_color, edge_color = "lightseagreen", "turquoise" # for core MOM6        
    elif "/FMS/" in source_path:
        node_color, edge_color = "orange", "gold"
        
    return node_color, edge_color

In [14]:
net = Network(notebook=True, directed=True, height='800px', cdn_resources='in_line')

net.set_options("""
var options = {
  "physics": { "enabled": false },
  "nodes": { "shape": "dot", "size": 10 },
  "edges": { "arrows": { "to": { "enabled": true, "scaleFactor": 0.3 }    } }
}
""")

# Add nodes with positions
for node in G.nodes():

    node_color, _ = get_color(G, node)

    x, y = pos[node]
    net.add_node(node, color=node_color, label=str(node), x=x, y=-y, fixed=True)  # y flipped for browser coord system

# Add edges
for source, target in G.edges():
    _, edge_color = get_color(G, source)
    net.add_edge(source, target, color=edge_color)

In [15]:
net.show("module_dependency.html")

module_dependency.html
